In [1]:
from Bio import SeqIO
import time
import psutil
import os

In [2]:
""" Paths to FASTA data files """

pathToFile1 = "./data/13443_ref_Cara_1.0_chr1c.fa"
pathToFile2 = "./data/10093_ref_PAHARI_EIJ_v1.1_chrX.fa"
pathToFile3 = "./data/144034_ref_Pbar_UMD_V03_chrUn.fa"

files = [
    pathToFile1,
    pathToFile2,
    pathToFile3
]

In [3]:
""" Search patterns for data files """

patternsForFile1 = [
    "ATGCATG",
    "TCTCTCTA",
    "TTCACTACTCTCA"
]

patternsForFile2 = [
    "ATGATG",
    "CTCTCTA",
    "TCACTACTCTCA"
]

patternsForFile3 = []

In [4]:
""" Return list of rotations of input string t """
def Rotations(t):
    tt = t * 2
    return [tt[i : i + len(t)] for i in range(0, len(t))]

In [5]:
""" Return lexicographically sorted list of t's rotations """
def BWM(t):
    return sorted(Rotations(t))

In [6]:
""" Given T, returns BWT(T) (last column) by creating BWM """
def BWTViaBWM(t):
    return ''.join(map(lambda x: x[-1], BWM(t)))

In [7]:
""" Test examples """

test1 = "Tomorrow_and_tomorrow_and_tomorrow$"
test2 = "It_was_the_best_of_times_it_was_the_worst_of_times$"
test3 = "in_the_jingle_jangle_morning_Ill_come_following_you$"
test4 = "GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTTGATTTGG\
GGTTCAAAGCAGTAATTTGGGGTTCAAAGCAGTATCGACAAATAGTAAATCCATTTGTTCATTCAAAGCAGTAATT\
TGGGGTTATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT$"

In [8]:
""" Test of Burrows-Wheeler Matrix algorithm """

print(BWTViaBWM(test1) + "\n")
print(BWTViaBWM(test2) + "\n")
print(BWTViaBWM(test3) + "\n")
print(BWTViaBWM(test4) + "\n")

w$wwdd__nnoooaattTmmmrrrrrrooo__ooo

s$esttssfftteww_hhmmbootttt_ii__woeeaaressIi_______

u_gleeeengj_mlhl_nnnnt$nwj__lggIolo_iiiiarfcmylo_oo_

TCCCCCCCCTTTCCAAAAAAAAAAATTGCCAAAAAAATTTCCCCCCCAAAGGAAATTTCAGATGCCCTTTTTTTATTTTGGGGGAATCCCTTTTTTAACCCT$AAAAATTTTTGGGGGGGGGGAAAAAAAAGGGGGTTTAATGGGGGAAAGGGTTTTTTAATTCCTAAAAAATTTTTTTTTTGAGGGGGGGTTTTTTTTTGGAAAAAAAA



In [9]:
""" Save Burrows-Wheeler transforms for later test of reversal function """

bwt1 = BWTViaBWM(test1)
bwt2 = BWTViaBWM(test2)
bwt3 = BWTViaBWM(test3)
bwt4 = BWTViaBWM(test4)

In [10]:
""" Given T return suffix array SA(T) """
def SuffixArray(s):
    satups = sorted([(s[i:], i) for i in range(len(s))])
    # Extract and return just the offsets
    return map(lambda x: x[1], satups)

In [11]:
""" Given T, returns BWT(T) (last column) by way of the suffix array """
def BWTViaSA(t):
    bw = []
    for si in SuffixArray(t):
        if si == 0:
            bw.append('$')
        else:
            bw.append(t[si - 1])
    return ''.join(bw) # returns string version of list bw

In [12]:
""" Test of Suffix Array algorithm """

print(BWTViaSA(test1) + "\n")
print(BWTViaSA(test2) + "\n")
print(BWTViaSA(test3) + "\n")
print(BWTViaSA(test4) + "\n")

w$wwdd__nnoooaattTmmmrrrrrrooo__ooo

s$esttssfftteww_hhmmbootttt_ii__woeeaaressIi_______

u_gleeeengj_mlhl_nnnnt$nwj__lggIolo_iiiiarfcmylo_oo_

TCCCCCCCCTTTCCAAAAAAAAAAATTGCCAAAAAAATTTCCCCCCCAAAGGAAATTTCAGATGCCCTTTTTTTATTTTGGGGGAATCCCTTTTTTAACCCT$AAAAATTTTTGGGGGGGGGGAAAAAAAAGGGGGTTTAATGGGGGAAAGGGTTTTTTAATTCCTAAAAAATTTTTTTTTTGAGGGGGGGTTTTTTTTTGGAAAAAAAA



In [13]:
""" Given BWT string bw, return parallel list of B-ranks. Also
    return tots: map from character to # times it appears. """
def RankBWT(bw):
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots:
            tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

In [14]:
""" Return map from character to the range of rows prefixed by 
    the character. """
def FirstColumn(tots):
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [15]:
""" Make T from BWT(T) """
def ReverseBWT(bw):
    ranks, tots = RankBWT(bw)
    first = FirstColumn(tots)
    rowi = 0   # first row
    t = '$'    # rightmost character
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t    # prepend to answer
        # jump to row that starts with c of same rank
        rowi = first[c][0] + ranks[rowi]
    return t

In [16]:
""" Test of Reversal algorithm """

assert test1 == ReverseBWT(bwt1)
assert test2 == ReverseBWT(bwt2)
assert test3 == ReverseBWT(bwt3)
assert test4 == ReverseBWT(bwt4)

In [17]:
""" Performance measurement and benchmarking """

%timeit BWTViaBWM(test1)
%timeit BWTViaBWM(test2)
%timeit BWTViaBWM(test3)
%timeit BWTViaBWM(test4)

print()

%timeit BWTViaSA(test1)
%timeit BWTViaSA(test2)
%timeit BWTViaSA(test3)
%timeit BWTViaSA(test4)

print()

%timeit ReverseBWT(bwt1)
%timeit ReverseBWT(bwt2)
%timeit ReverseBWT(bwt3)
%timeit ReverseBWT(bwt4)

14 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
24.9 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
23.5 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
91.8 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

20 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
28.5 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
29.7 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
136 µs ± 2.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

23.9 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
36 µs ± 3.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
35.9 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
109 µs ± 4.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
!python "./memTest/bwmTest.py" $test1
!python "./memTest/bwmTest.py" $test2
!python "./memTest/bwmTest.py" $test3
!python "./memTest/bwmTest.py" $test4

Used this much memory: 13.703125 Mb
Used this much memory: 13.75 Mb
Used this much memory: 13.81640625 Mb
Used this much memory: 13.859375 Mb


In [19]:
!python "./memTest/saTest.py" $test1
!python "./memTest/saTest.py" $test2
!python "./memTest/saTest.py" $test3
!python "./memTest/saTest.py" $test4

Used this much memory: 13.73828125 Mb
Used this much memory: 13.76171875 Mb
Used this much memory: 13.78125 Mb
Used this much memory: 13.80078125 Mb


In [20]:
!python "./memTest/reverseTest.py" $bwt1
!python "./memTest/reverseTest.py" $bwt2
!python "./memTest/reverseTest.py" $bwt3
!python "./memTest/reverseTest.py" $bwt4

Used this much memory: 13.875 Mb
Used this much memory: 13.84765625 Mb
Used this much memory: 13.8359375 Mb
Used this much memory: 13.78515625 Mb


In [21]:
""" FASTA sequences are often broken in smaller chunks. This function
    appends all sequences for given file. """
def GetWholeSequenceFromFile(file):
    # all genome records of given FASTA file
    records = list(SeqIO.parse(file, "fasta"))
    sequence = ""
    # iterate over each record element
    for i in range(0, len(records)):
        sequence += records[i].seq
    # sequence is now whole
    return "".join(str(sequence).split())

In [22]:
sequences = [] # list containing all of the sequences we need to work with

for file in files:
    sequences.append(GetWholeSequenceFromFile(file) + '$')

In [23]:
#for i in range(1, len(sequences) + 1):
#    with open("./data/" + "sequence" + str(i) + ".txt", "w") as f:
#        f.write(sequences[i - 1])
#        f.close()

In [13]:
!python "./memTest/bwmTestFiles.py" 1
!python "./memTest/bwmTestFiles.py" 2
!python "./memTest/bwmTestFiles.py" 3

50001
Used this much memory: 91.45703125 Mb
50001
Used this much memory: 186.3984375 Mb
50001
Used this much memory: 270.43359375 Mb


In [10]:
!python "./memTest/saTestFiles.py" 1
!python "./memTest/saTestFiles.py" 2
!python "./memTest/saTestFiles.py" 3

Used this much memory: 92.86328125 Mb
Used this much memory: 186.89453125 Mb
Used this much memory: 268.8359375 Mb
